In [ ]:
# Imports 
import os
from os import listdir
import random
import itertools
from pathlib import Path

In [ ]:
# Main parameters
# change to our own directory with INRIA Aerial Image Dataset
inria_dataset_root_dir = Path("/media/dlsupport/DATA1/EOData/INRIA/AerialImageDataset")

# Define INRIA Dataset dataframe

The INRIA Aerial Image Labeling Dataset has the following structure : 

<!-- language: lang-none -->

    .
    ├── test
    │   └── images
    └── train
        ├── gt
        └── images
        
Each image has a filename of type {town_prefix}{i}.tif with i in [1:36] Gt (ground truth) and image has the same filename.
First we load info of train images with *load_geo_img_dir* utils function of EOTorchLoader

In [ ]:
import pandas as pd
from eotorchloader.dataset.utils import load_geo_img_dir

In [ ]:
inria_train_val_df = load_geo_img_dir(inria_dataset_root_dir/"train"/"images")

In [ ]:
inria_train_val_df.head(10)

then we define a split in train data between train and val dataset. This could be done in multiple way.

 * set val image as first images of each town. Usually for INRIA dataset val images are set as the 6 first image of each towns.
 * set val as all the image in a town and train as the images in all other towns.
 
To represent theses two splits we add 2 columns to the extracted dataframe :
 
 * a column "town" with the town id/name
 * a column "standart_split" with take value in ["train", "val", "test"]


In [ ]:
# First add a town column.
# This is done by splitting the name string values (as "tyrol-w28") in 2 parts : a name with alphabetic character ("tyrol-w") and a id/num with numerical values (28)
inria_train_val_df[['town', 'num']] = inria_train_val_df["name"].str.extract('([a-zA-Z\-]+)([^a-zA-Z\-]+)', expand=True)
# convert num from string to int
inria_train_val_df['num'] = inria_train_val_df['num'].astype(int)

# Next we add standard_split columns
inria_train_val_df["standard_split"] = "train" # first initialize all row with train
inria_train_val_df.loc[inria_train_val_df["num"]<=6 ,"standard_split"] = "val" # set all row/image with num < 6 as validation data

# finally we rename path as img_path and add a gt_path columns with corresponding mask path
inria_train_val_df = inria_train_val_df.rename(columns={"path": "img_path"})
inria_train_val_df["msk_path"] =  inria_train_val_df["img_path"].str.replace("images", "gt", regex=False)

In [ ]:
print(inria_train_val_df[['name','town', 'num', 'standard_split']]) 
print(inria_train_val_df['town'].unique())

Once we have the list of image and mask we could intialize a TorchDataset which crop the image.

 * the tile_size is set in pixel
 * by default no transofmr is apply and the sample are in form {"image" : np.array, "mask" :np.array } in channel first order (CHW or rasterio like)

In [ ]:
# import for use in train code
from eotorchloader.dataset.scene_dataset import LargeImageDataset

In [ ]:
inria_train_df = inria_train_val_df[inria_train_val_df["standard_split"]=="train"]
image_files_train = inria_train_df["img_path"].values
mask_files_train = inria_train_df["msk_path"].values
print(image_files_train[0:5])
print(mask_files_train[0:5])

train_dataset_tile = LargeImageDataset(
    image_files=image_files_train,
    mask_files=mask_files_train,
    tile_size = 512,
    transforms=None,
    image_bands=[1,2,3],
    mask_bands=[1])

In [ ]:
test_idx = 201
test_data = train_dataset_tile[test_idx]
print(f" keys : {test_data.keys()}")
img_shape =  test_data['image'].shape
msk_shape = test_data['mask'].shape
print(f" image shape : {img_shape}, mask shape : {msk_shape}")
print(f" mask type : {test_data['mask'].dtype}")

To display a sample or batch we could use some display function of EoTorchloader. 
As sample are not transformed from original format, to display mask a lut/nomenclature could be useful.

First we need to check what value are present in a mask array. In INRIA case it should be 

 * 0 for no bati
 * 255 for bati

In [ ]:
import numpy as np
print(np.histogram(test_data['mask'], bins=10))
print(np.unique(test_data['mask']))

### configure tranform for training

for pytorch training we need to scale input image and mask to [0-1] range and to convert to float tensor.

In [ ]:
from eotorchloader.transform.tensor import ToTorchTensor
from eotorchloader.transform.scale import ScaleImageToFloat

In [ ]:
# transforms for training
inria_train_tf = [
    ScaleImageToFloat(scale_factor=255, clip=True, img_only=False),
    ToTorchTensor()
]

train_dataset_tile_b = LargeImageDataset(
    image_files=image_files_train,
    mask_files=mask_files_train,
    tile_size = 384,
    transforms=inria_train_tf,
    image_bands=[1,2,3],
    mask_bands=[1])

In [ ]:
test_idx = 240
test_data = train_dataset_tile_b[test_idx]
print(f" keys : {test_data.keys()}")
img_shape =  test_data['image'].shape
msk_shape = test_data['mask'].shape
print(f" image shape : {img_shape}, mask shape : {msk_shape}")
print(f" mask type : {test_data['mask'].dtype}")

### use transform to display sample

In [ ]:
inria_lut = np.array([
 [  0, 255,  255, 255], # white 
 [  255, 255, 50, 150]  # pink
])


In [ ]:
# display import
from eotorchloader.transform.tensor import CHW_to_HWC
from eotorchloader.transform.display import ToRgbDisplay

from eotorchloader.display.matplotlib import view_patch, view_batch

In [ ]:
channel_last_transform = CHW_to_HWC(img_only=True)
display_patch_transform = ToRgbDisplay(lut=inria_lut, flatten_mask=False)

In [ ]:
test_data = train_dataset_tile[142]
view_patch(test_data, transforms=[channel_last_transform, display_patch_transform])